## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,59.00,77,8,5.75,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,51.80,82,75,10.36,broken clouds
2,2,Nebbi,UG,2.4783,31.0889,69.67,27,100,4.74,overcast clouds
3,3,Sisimiut,GL,66.9395,-53.6735,24.67,79,2,17.05,clear sky
4,4,Emba,KZ,48.8264,58.1442,-5.13,90,30,12.53,scattered clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Vaini,TO,-21.2000,-175.2000,77.00,94,90,16.11,moderate rain
8,8,Merauke,ID,-8.4667,140.3333,84.88,73,43,11.79,scattered clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,77.00,96,100,1.99,light rain
23,23,Hilo,US,19.7297,-155.0900,80.60,54,20,12.66,few clouds
25,25,Kavieng,PG,-2.5744,150.7967,85.10,70,8,7.20,clear sky
29,29,Sawtell,AU,-30.3833,153.1000,78.80,61,40,17.27,scattered clouds
31,31,Butaritari,KI,3.0707,172.7902,80.94,80,18,18.84,few clouds
32,32,Hithadhoo,MV,-0.6000,73.0833,81.61,72,0,8.16,clear sky
34,34,Saint-Pierre,RE,-21.3393,55.4781,80.60,61,90,9.22,overcast clouds
36,36,Bambous Virieux,MU,-20.3428,57.7575,78.80,83,40,6.91,light rain


In [18]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.isnull()
clean_df.isnull().sum()
# or use preferred_cities_df.notnull().sum()
# else use preferred_cities_df.count()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#If you have some rows with null values, you'll need to drop them using the dropna() method 
#at the end of your filtering statement when you are creating the new DataFrame.
#preferred_cities_df.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Vaini,TO,77.00,moderate rain,-21.2000,-175.2000,
8,Merauke,ID,84.88,scattered clouds,-8.4667,140.3333,
9,Puerto Ayora,EC,77.00,light rain,-0.7393,-90.3518,
23,Hilo,US,80.60,few clouds,19.7297,-155.0900,
25,Kavieng,PG,85.10,clear sky,-2.5744,150.7967,
29,Sawtell,AU,78.80,scattered clouds,-30.3833,153.1000,
31,Butaritari,KI,80.94,few clouds,3.0707,172.7902,
32,Hithadhoo,MV,81.61,clear sky,-0.6000,73.0833,
34,Saint-Pierre,RE,80.60,overcast clouds,-21.3393,55.4781,
36,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        hotel_df
    except (IndexError):
        print("Hotel not found... skipping.")
        hotels
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [26]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Vaini,TO,77.00,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
8,Merauke,ID,84.88,scattered clouds,-8.4667,140.3333,
9,Puerto Ayora,EC,77.00,light rain,-0.7393,-90.3518,Castillo Galapagos
23,Hilo,US,80.60,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
25,Kavieng,PG,85.10,clear sky,-2.5744,150.7967,Nusa Island Retreat
...,...,...,...,...,...,...,...
666,Calamar,CO,80.60,scattered clouds,10.2511,-74.9146,Hostal Katyalex
668,Mogadishu,SO,77.25,clear sky,2.0371,45.3438,Hotel Juba
671,Maceio,BR,77.00,clear sky,-9.6658,-35.7353,Hotel Ponta Verde Maceió
673,Tagbac,PH,77.00,overcast clouds,7.0890,125.1567,Sitio Maupot Family Resort


In [27]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
#Description = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))